In [1]:
import sys
import time
from pyqtgraph.Qt import QtCore, QtGui
import numpy as np
import pyqtgraph as pg


class App(QtGui.QMainWindow):
    def __init__(self, parent=None):
        super(App, self).__init__(parent)

        #### Create Gui Elements ###########
        self.mainbox = QtGui.QWidget()
        self.setCentralWidget(self.mainbox)
        self.mainbox.setLayout(QtGui.QVBoxLayout())

        self.canvas = pg.GraphicsLayoutWidget()
        self.mainbox.layout().addWidget(self.canvas)

        self.label = QtGui.QLabel()
        self.mainbox.layout().addWidget(self.label)

        self.view = self.canvas.addViewBox()
        self.view.setAspectLocked(True)
        self.view.setRange(QtCore.QRectF(0,0, 100, 100))

        #  image plot
        self.img = pg.ImageItem(border='w')
        self.view.addItem(self.img)

        self.canvas.nextRow()
        #  line plot
        self.otherplot = self.canvas.addPlot()
        self.h2 = self.otherplot.plot(pen='y')


        #### Set Data  #####################

        self.x = np.linspace(0,50., num=100)
        self.X,self.Y = np.meshgrid(self.x,self.x)

        self.counter = 0
        self.fps = 0.
        self.lastupdate = time.time()

        #### Start  #####################
        self._update()

    def _update(self):

        self.data = np.sin(self.X/3.+self.counter/9.)*np.cos(self.Y/3.+self.counter/9.)
        self.ydata = np.sin(self.x/3.+ self.counter/9.)

        self.img.setImage(self.data)
        self.h2.setData(self.ydata)

        now = time.time()
        dt = (now-self.lastupdate)
        if dt <= 0:
            dt = 0.000000000001
        fps2 = 1.0 / dt
        self.lastupdate = now
        self.fps = self.fps * 0.9 + fps2 * 0.1
        tx = 'Mean Frame Rate:  {fps:.3f} FPS'.format(fps=self.fps )
        self.label.setText(tx)
        QtCore.QTimer.singleShot(1, self._update)
        self.counter += 1


app = QtGui.QApplication(sys.argv)
thisapp = App()
thisapp.show()
sys.exit(app.exec_())

C:\Users\mgrinberg\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


SystemExit: 0

C:\Users\mgrinberg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import pyqtgraph as pg
import numpy as np
from pyqtgraph.Qt import QtCore, QtGui
import random


app = QtGui.QApplication([])


plt = pg.plot()
bufferSize = 2000
maxTime = 3
data = np.zeros((bufferSize, 2))

curve = plt.plot()
line = plt.addLine(x=0)
plt.setRange(xRange=[0, maxTime], yRange=[-50, 50])
i = 0
lasti = 0
n_times = 0 
t_start = pg.ptime.time()

def update():
    global data, curve, line, i, t_start, lasti, n_times
    n = 1  # update 10 samples per iteration
    rand = random.randint(-1,1)
    #print rand
    dt = pg.ptime.time()-t_start
    data[i+n, 0] = np.clip(data[i, 0] + rand, -50, 50)
    data[i+n, 1] = dt
    i = i+n
    n_times += 1
    #print i
    if dt >= 3:
        t_start = pg.ptime.time()
        print n_times
        n_times = 0
        data[i]
        i = 0
    line.setValue(dt)
    curve.setData(data[:,1],data[:,0])

        
timer = pg.QtCore.QTimer()
timer.timeout.connect(update)
timer.start(0) #Hopefully makes it run as fast as possible.


if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(print n_times)? (<ipython-input-1-7141600ce449>, line 36)

In [ ]:
#https://stackoverflow.com/questions/32303045/what-is-most-efficient-python-ipc-mechanism-for-plotting-real-time-serial-data

from pyqtgraph.Qt import QtGui
import pyqtgraph as pg
from multiprocessing import Process, Array, Value, Lock
from serial_interface import EEG64Board
from collections import deque

def serialLoop(arr,idx,lock):
    eeg = EEG64Board(port='/dev/ttyACM0')
    eeg.openSerial() 
    eeg.sendTest('1')        #Tells the eeg device to start sending data
    while True:
        data = eeg.readEEG() #Returns an array of the 8 latest values, one per channel
        if data != False:    #Returns False if bad checksum
            lock.acquire()
            for i in range(8):
                arr[i][idx.value] = data[i] 
            idx.value += 1
            lock.release()
    eeg.sendTest('2') 

arr = [Array('d',range(1024)) for i in range(8)]
idx = Value('i', 0)
q = [deque([],500) for i in range(8)]
iq = deque([],500)
lock = Lock()

lastUpdate = pg.ptime.time()
avgFps = 0.0

def graphLoop():
    global val,q,lock,arr,iq, lastUpdate, avgFps
    win = pg.GraphicsWindow()
    plt = list()
    for i in range(8):
        plt += [win.addPlot(row=(i+1), col=0, colspan=3)]
    #iplt = pg.plot(iq)
    counter = 0
    while True:
        lock.acquire()
        #time.sleep(.01)
        for i in range(idx.value):
            for j in range(8):
                q[j].append(arr[j][i])        
        idx.value = 0
        lock.release()
        for i in range(8):
            plt[i].plot(q[i],clear=True)
        QtGui.QApplication.processEvents()
        counter += 1

        now = pg.ptime.time()
        fps = 1.0 / (now - lastUpdate)
        lastUpdate = now
        avgFps = avgFps * 0.8 + fps * 0.2

serial_proc = Process(target=serialLoop, args=(arr,idx,lock), name='serial_proc')
serial_proc.start()

graphLoop()

serial_proc.terminate()

In [ ]:
https://stackoverflow.com/questions/32303045/what-is-most-efficient-python-ipc-mechanism-for-plotting-real-time-serial-data

In [ ]:
https://github.com/pyqtgraph/pyqtgraph/blob/develop/examples/Plotting.py
http://www.pyqtgraph.org/images/plotting.png

In [2]:
https://groups.google.com/forum/#!msg/pyqtgraph/haiJsGhxTaQ/sTtMa195dHsJ
https://groups.google.com/forum/#!topic/pyqtgraph/H6SSfbUneds